# Interactive Plotting of Open Flight Data with Datashader

## TODO
* Determine if parq file causes plot to not be interactive
* Plot full day of data
* Try with short trail days
* Use new agg files with categories and use for coloring

In [1]:
import bokeh
import bokeh.plotting as plotting
from bokeh.models import WMTSTileSource
plotting.output_notebook()
import datashader.transfer_functions as tf
import datashader as ds
from datashader.colors import viridis
from datashader.bokeh_ext import InteractiveImage
from datashader.utils import lnglat_to_meters as webm
import dask
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
ProgressBar().register()
import pandas as pd
import numpy as np
np.warnings.filterwarnings('ignore')
from sklearn.externals.joblib import Memory
memory = Memory(location='/tmp', verbose=0)

Loading BokehJS ...

In [2]:
print(f'Pandas version: {pd.__version__}')
print(f'Bokeh version: {bokeh.__version__}')
print(f'Datashader version: {ds.__version__}')
print(f'dask version: {dask.__version__}')

Pandas version: 0.23.4
Bokeh version: 0.13.0
Datashader version: 0.6.8
dask version: 0.18.2


In [3]:
%%time
parq_file = '/users/lukestarnes/Documents/adsb/2018.parq'
h5_file = '/users/lukestarnes/Documents/adsb/2018-06-19.h5'
# ddf = dd.read_parquet(parq_file)
ddf = pd.read_hdf(h5_file, stop=2_000_000, columns=['Lat', 'Long'])

CPU times: user 46.8 s, sys: 9.85 s, total: 56.6 s
Wall time: 59.6 s


In [4]:
w = webm(ddf.Long, ddf.Lat)
ddf.loc[:,'x'] = w[0]
ddf.loc[:,'y'] = w[1]

In [5]:
MaxBounds = ((-20048966.10, 20048966.10), (-20026376.39, 20026376.39))
WholeWorld = ((-20_037_508, 20_037_508), (-7_670_608, 13_971_466))
TwoBounds = ((-20_000_000, 20_000_000), (-20_000_000, 20_000_000))
USA_CONUS = ((-13884029, -7453304), (2698291, 6455972))
WesternEuro = ((-1181114, 4270391), (3000000, 8081620))
Germany = ((709336, 1600000), (6026907, 7270000))
Chicago = (( -9828281, -9717659), (5096658, 5161298))
Chinatown = (( -9759210, -9754583), (5137122, 5139825))
NewYorkCity = (( -8280656, -8175066), (4940514, 4998954))
LosAngeles = ((-13195052, -13114944), (3979242, 4023720))
Houston = ((-10692703, -10539441), (3432521, 3517616))
Austin = ((-10898752, -10855820), (3525750, 3550837))
NewOrleans = ((-10059963, -10006348), (3480787, 3510555))
Atlanta = ((-9507853,-9274873), (3927030, 4069506))

In [6]:
# source: https://leaflet-extras.github.io/leaflet-providers/preview/
Esri_NatGeoWorldMap = 'https://server.arcgisonline.com/ArcGIS/rest/services/NatGeo_World_Map/MapServer/tile/{z}/{y}/{x}'
Esri_OceanBasemap = 'https://server.arcgisonline.com/ArcGIS/rest/services/Ocean_Basemap/MapServer/tile/{z}/{y}/{x}'
CartoDB_Positron = 'https://cartodb-basemaps-{s}.global.ssl.fastly.net/light_all/{z}/{x}/{y}{r}.png'
CartoDB_Voyager = 'https://cartodb-basemaps-{s}.global.ssl.fastly.net/rastertiles/voyager/{z}/{x}/{y}{r}.png'
OpenStreetMap_Mapnik = 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png'
OpenTopoMap = 'https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png'
Hydda_Full = 'https://{s}.tile.openstreetmap.se/hydda/full/{z}/{x}/{y}.png'
Esri_WorldStreetMap = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}'
Esri_WorldTopoMap = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}'
Esri_WorldImagery = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'

In [7]:
def spread(pts):
    return ((pts[0][1] - pts[0][0]),
            (pts[1][1] - pts[1][0]))
def ratio(pts):
    s = spread(pts)
    x, y = s
    return x / y

In [15]:
def base_plot(xrange, yrange, plot_width=int(900), plot_height=int(500),
              tools='pan,wheel_zoom,zoom_in,zoom_out,reset', 
              bok_cir = True, tile_url=CartoDB_Positron):
    p = plotting.figure(tools=tools,
                  plot_width=plot_width, plot_height=plot_height,
                  x_range=(xrange), y_range=yrange, outline_line_color=None,
                  min_border=0, min_border_left=0, min_border_right=0,
                  min_border_top=0, min_border_bottom=0)
    p.match_aspect = True
    p.axis.visible = False
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    if bok_cir:
        p.circle(x="x", y="y",color='red', size=2, alpha=0.4) 
    tile_renderer = p.add_tile(WMTSTileSource(url=tile_url)) 
    tile_renderer.alpha=0.8
    return p

In [16]:
@memory.cache
def create_image(x_range, y_range, plot_weight, plot_height, thresh=None):
    cmap=viridis
    cvs = ds.Canvas(plot_weight, plot_height, x_range, y_range)
    agg = cvs.points(ddf, 'x', 'y')
    if thresh is not None:
        img = tf.shade(agg.where(agg > thresh), cmap = cmap)
    else:
        img = tf.shade(agg, cmap = cmap)
    return img

def image_callback(xr, yr, w, h):
    return create_image(xr, yr, w, h, thresh=thold)

In [17]:
thold = None
p = base_plot(*WholeWorld, bok_cir=False, tile_url=Esri_NatGeoWorldMap)
InteractiveImage(p, image_callback)